In [ ]:
import pandas as pd
import numpy as np
import pickle

In [16]:
with open(r'C:\Users\rgarlay\Desktop\DS\Churn Project\Churn-Prediction\final_obj\model.pkl','rb') as file:
    model = pickle.load(file)

In [19]:
test_file = pd.read_csv(r'C:\Users\rgarlay\Desktop\DS\Churn Project\Churn-Prediction\Notebooks\test.csv')

In [20]:
test_file.head()

,customer_id,age,location,subscription_type,payment_plan,num_subscription_pauses,payment_method,customer_service_inquiries,signup_date,weekly_hours,average_session_length,song_skip_rate,weekly_songs_played,weekly_unique_songs,num_favorite_artists,num_platform_friends,num_playlists_created,num_shared_playlists,notifications_clicked
0,200000,48,Virginia,Free,Monthly,1,Debit Card,High,-1644,22.206067,43.264881,0.255031,299,165,12,155,42,27,45
1,200001,47,North Dakota,Premium,Monthly,0,Debit Card,High,-2718,41.473683,70.155977,0.978803,437,196,42,16,4,1,4
2,200002,25,North Dakota,Free,Monthly,1,Credit Card,Medium,-2783,16.565634,24.965464,0.569179,57,236,5,108,38,44,45
3,200003,72,Montana,Student,Monthly,0,Debit Card,Low,-757,21.805222,21.747876,0.989232,181,245,32,11,80,8,39
4,200004,71,California,Family,Monthly,4,Debit Card,Low,-80,33.561459,7.874533,0.898800,348,31,8,174,67,6,1


In [ ]:
COLUMNS_TO_DROP =  ['location','payment_method',
                            'customer_id','payment_plan',
                            'signup_date','average_session_length',
                          'num_favorite_artists','num_playlists_created',
                          'weekly_songs_played', 'weekly_unique_songs']

In [ ]:
test_file.drop(columns=COLUMNS_TO_DROP, inplace=True)

In [ ]:


def df_transform(x: pd.DataFrame, mapping:dict):

    x_copy = x.copy()

    # ----- age group -----
    bins = [18, 25, 35, 45, 55, 65, 80]
    labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65-79']
    x_copy['age_group'] = pd.cut(
        x_copy['age'],
        bins=bins,
        labels=labels,
        right=False
    )

    # ----- notification segment -----
    x_copy['notif_segment'] = pd.cut(
        x_copy['notifications_clicked'],
        bins=[-1, 4, 44, float('inf')],
        labels=['Non-Engager', 'Standard', 'Extreme']
    )

    x_copy['extrovertness_index'] = (
        x_copy['num_shared_playlists'] /
        (x_copy['num_platform_friends'] + 1)
    )

    for col, func in mapping.items():
            x_copy[col] = x_copy[col].map(func)

    x_copy.drop(
        columns=[
            'age',
            'notifications_clicked',
            'num_shared_playlists',
            'num_platform_friends'
        ],
        inplace=True
    )

    return x_copy

DATA_TRANSFORMATION_FEATURE_MAPPING = {'customer_service_inquiries':
                                                    {'Low': 0, 'Medium': 1, 'High': 2},
                                                'subscription_type':
                                                    {'Free': 0, 'Student': 1, 'Family': 2, 'Premium': 3},
                                                'notif_segment':
                                                    {'Non-Engager': 0, 'Standard': 1, 'Extreme': 2},
                                                'age_group':
                                                    {'18-24': 2,'65-79': 2,'25-34': 1,
                                                    '55-64': 1,'35-44': 0,'45-54': 0 }}

In [ ]:
tet_file = df_transform(x=test_file, mapping = DATA_TRANSFORMATION_FEATURE_MAPPING)

In [36]:
tet_file.head()

,subscription_type,num_subscription_pauses,customer_service_inquiries,weekly_hours,song_skip_rate,age_group,notif_segment,extrovertness_index
0,0,1,2,22.206067,0.255031,0,2,0.173077
1,3,0,2,41.473683,0.978803,0,0,0.058824
2,0,1,1,16.565634,0.569179,1,2,0.403670
3,1,0,0,21.805222,0.989232,2,1,0.666667
4,2,4,0,33.561459,0.898800,2,0,0.034286


In [39]:
with open(r'C:\Users\rgarlay\Desktop\DS\Churn Project\Churn-Prediction\final_obj\preprocessor.pkl','rb') as file:
    preprocessor = pickle.load(file)
    

In [40]:
preprocessed_df = preprocessor.transform(tet_file)

In [45]:
predictions = model.predict_proba(preprocessed_df)

In [46]:
predictions_1 = predictions.copy()

In [48]:
preds_1 = predictions_1[:,-1]

#### People above 0.80 are taken to be high-risk of churn. Let's count them.

In [51]:
high_risk = {'High_Risk':0,'Not_High_Risk':0}
for element in preds_1:
    if element > 0.80:
        high_risk['High_Risk'] +=1
    else:
        high_risk['Not_High_Risk'] +=1 

In [56]:
high_risk

{'High_Risk': 22251, 'Not_High_Risk': 52749}